In [9]:
# Imports
import torch
from dask_jobqueue import SLURMCluster
from distributed import Client
from run_flows_mixed_bd import run_experiment
from csuite_mixed import SCMS

In [10]:
seeds = 10
nsamples = 2000
corr = 0.0
use_dag = False
affine = False
consistent = False
#scms = ["generate_backdoor_linear", "generate_backdoor_nonlinear"]
scms = list(SCMS.keys())

In [11]:
cluster = SLURMCluster(
    n_workers=0,
    memory="20GB",
    processes=1,
    cores=1,
    scheduler_options={
        "dashboard_address": ":10092",
        "allowed_failures": 10
    },
    job_cpu=1,
    walltime="24:0:0",
)
cluster.adapt(minimum=0, maximum=200)
client = Client(cluster)

In [12]:
scms

['2var_linear',
 '2var_nonlinear',
 'triangle_linear',
 'triangle_nonlinear',
 'fork_linear',
 'fork_nonlinear',
 'chain5_linear',
 'chain5_nonlinear']

In [13]:
futures = []
for scm in scms:
    for seed in range(seeds):
        f = client.submit(run_experiment, 
                          sc_name = scm,
                          seed = seed,
                          use_dag = use_dag,
                          corr = corr,
                          N = nsamples,
                          affine = affine,
                          consistent = consistent)
        futures += [f] 

In [14]:
results = client.gather(futures)

In [15]:
client.close()
cluster.close()

2025-08-04 21:43:23,997 - distributed.deploy.adaptive_core - INFO - Adaptive stop


In [16]:
torch.save(f = "causalflow_csuite_mixed_bd_results_n={0}_corr={1}_dag={2}_affine={3}_consistent={4}_trials={5}.pt".format(nsamples, corr, use_dag, affine, consistent, seeds), obj = results)